In [80]:
pip install xlrd

In [178]:
#---------------------------------------------------------------------------------------------------------------
# Nom du projet     : Classification
# Nom du programme  : SDA_Projet_Classification                                                                         
# Description       : L’objectif est de prédire le parti gagnant des élections présidentielles de 2020 aux Etats
#                   : Unis à partir de données socio-démographiques                        
# Auteur            : Yaya KONE,Papa Moussa THIOUNE                                                                         
# Date de création  : Novembre 2022                       
#---------------------------------------------------------------------------------------------------------------

# Librairies utiles pour le data management
import pandas as pd 
import numpy as np
import datetime as dt           
import os, glob, re  

#---------------------------------------------------------------------------------------------------------------
# Partie 1 : Constitution des donnes
#---------------------------------------------------------------------------------------------------------------

# Importation des resultats
df_results_2020 = pd.read_csv('data/2020_US_County_Level_Presidential_Results.csv')
df_results_0816 = pd.read_csv('data/US_County_Level_Presidential_Results_08-16.csv')

# Creation de la variable target dans la base df_results_2020
df_results_2020['target'] = df_results_2020['diff'].apply(lambda x : 1 if x>0 else 0)

# lecture des fichiers xls et concatenation des donnees socio-demographiques
list_file_xls = glob.glob("data/*.xls")              # Liste des fichiers xls contenus dans le dossier data
df_socio = pd.read_excel(list_file_xls[0], header=4) # Lecture du 1er fichier de la liste 
df_socio = df_socio.set_index(df_socio.columns[0])   # On utilise la variable fips comme index
for i in range(1,len(list_file_xls)):
    if 'Population' in list_file_xls[i]:
        df = pd.read_excel(list_file_xls[i], header=2) 
    else: 
        df = pd.read_excel(list_file_xls[i], header=4) 
    df = df.set_index(df.columns[0])
    df_socio = pd.concat([df_socio, df], axis=1) 

# Creation de la base df_votes contenant les resultats de 2008 a 2016 et les donnees socio-demographiques
df_votes = df_results_0816.set_index(['fips_code'])
df_votes = df_votes.join(df_socio)

# Affichage des bases de donnees df_votes
print(f"La base df_votes fait {df_votes.shape[0]} lignes et {df_votes.shape[1]} colonnes\n") # On affiche le nombre de lignes et de colonnes 
print(f"{df_votes.head()}\n")                                                                # On affiche les 5 premieres lignes 

La base df_votes fait 3112 lignes et 343 colonnes

                       county  total_2008  dem_2008  gop_2008  oth_2008  \
fips_code                                                                 
26041            Delta County       19064      9974      8763       327   
48295         Lipscomb County        1256       155      1093         8   
1127            Walker County       28652      7420     20722       510   
48389           Reeves County        3077      1606      1445        26   
56017      Hot Springs County        2546       619      1834        93   

           total_2012  dem_2012  gop_2012  oth_2012  total_2016  ...  \
fips_code                                                        ...   
26041           18043      8330      9533       180       18467  ...   
48295            1168       119      1044         5        1322  ...   
1127            28497      6551     21633       313       29243  ...   
48389            2867      1649      1185        33        3184

In [182]:
#---------------------------------------------------------------------------------------------------------------
# Partie 2 : L’analyse exploratoire
#---------------------------------------------------------------------------------------------------------------

# Quelques informations sur les bases de données train_val et test
print("Les informations sur la base df_votes")
print(f"{df_votes.info()}\n")                  
print(f"{df_votes.describe(include='all')}\n")  

#---------------------------------------------------------------------------------------------------------------
# Partie 2.1 : Suppression des variables non pertinentes au regard du dictionnaire de donnnes
#---------------------------------------------------------------------------------------------------------------

# Les totaux n'ont pas un grand interet a etre modelise car ils seront fortement correlles au nombre de votants dem et gop 
# Les codes ne sont pas caracteristiques des donnees socio-demographiques 
col_to_delete_1 = [col for col in df_votes.columns if 'total' in col.lower()]
col_to_delete_2 = [col for col in df_votes.columns if 'code' in col.lower()]
list_col_to_delete = col_to_delete_1 + col_to_delete_2
df_votes.drop(columns = list_col_to_delete, inplace = True)

#---------------------------------------------------------------------------------------------------------------
# Partie 2.1 : Traitement des doublons 
#---------------------------------------------------------------------------------------------------------------

print(f"La base df_votes contient {df_votes[df_votes.duplicated(keep=False)].shape[0]} vrais doublons")


Les informations sur la base df_votes
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3112 entries, 26041 to 20063
Columns: 325 entries, county to Percent of adults with a bachelor's degree or higher, 2015-19
dtypes: float64(307), int64(9), object(9)
memory usage: 7.8+ MB
None

                   county      dem_2008       gop_2008      oth_2008  \
count                3112  3.112000e+03    3112.000000   3112.000000   
unique               1845           NaN            NaN           NaN   
top     Washington County           NaN            NaN           NaN   
freq                   30           NaN            NaN           NaN   
mean                  NaN  2.231125e+04   19207.531491    587.632391   
std                   NaN  7.715532e+04   44866.795361   1846.444842   
min                   NaN  8.000000e+00      67.000000      0.000000   
25%                   NaN  1.806250e+03    2881.250000     70.000000   
50%                   NaN  4.499000e+03    6312.000000    169.000000   

In [ ]:
# Ces huit variables serviront à identifier la presence de doublons
features_analyze_duplicates = []